## **Libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score, precision_score, recall_score

In [6]:
cqt_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/cqt/00_BN1-129-Eb_comp_mix_cqt.npy"
X = np.load(cqt_path)  # shape: (6, T, 144)
print("✅ CQT shape:", X.shape)

✅ CQT shape: (6, 962, 144)


In [9]:
json_path = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet/merge_annotation/00_BN1-129-Eb_solo_mix_fretnet.json"

with open(json_path, 'r') as f:
    data = json.load(f)

num_strings = 6
num_frets = 19  # fret 0–18 + 1 (open)
frame_count = len(data["times"])

# 🔢 Y çıkışı: frame × (6 × fret)
Y = np.zeros((frame_count, num_strings * (num_frets + 1)), dtype=np.float32)

# 🎼 Label doldurma (tablature: her string için fret aktif mi?)
tablature = np.array(data["tablature"]).T

for t in range(frame_count):
    for s in range(num_strings):
        f = tablature[t][s]
        if f >= 0 and f <= num_frets:
            idx = s * (num_frets + 1) + int(f)
            Y[t, idx] = 1.0

print("✅ Label shape:", Y.shape)

✅ Label shape: (1044, 120)


In [8]:
print(type(data["tablature"]))
print(np.array(data["tablature"]).shape)
print(data["tablature"][:3])  # ilk 3 frame örneği

<class 'list'>
(6, 1044)
[[-1, 6, 6, 6, 6, 6, 6, 6, 6, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -

## **Dataset**

In [10]:
class TabCNNDataset(Dataset):
    def __init__(self, X, Y, context_size=9):
        # Frame sayısını en küçüğe göre kes
        min_len = min(X.shape[1], Y.shape[0])
        self.X = X[:, :min_len, :]
        self.Y = Y[:min_len]

        self.context_size = context_size
        self.pad = context_size // 2
        self.T = min_len

        # Zaman ekseninde padding
        self.X_padded = np.pad(self.X, ((0, 0), (self.pad, self.pad), (0, 0)), mode='edge')

    def __len__(self):
        return self.T

    def __getitem__(self, idx):
        x = self.X_padded[:, idx:idx+self.context_size, :]
        y = self.Y[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [21]:
dataset = TabCNNDataset(X, Y, context_size=9)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for xb, yb in loader:
    print("X batch:", xb.shape)  # [B, 6, 9, 144]
    print("Y batch:", yb.shape)  # [B, 120]
    break

X batch: torch.Size([32, 6, 9, 144])
Y batch: torch.Size([32, 120])


## **Model**

In [12]:
class TabCNN(nn.Module):
    def __init__(self, num_strings=6, num_frets=19):
        super(TabCNN, self).__init__()
        self.num_outputs = num_strings * (num_frets + 1)

        # Input shape: (B, 6, 9, 144)
        self.conv1 = nn.Conv2d(6, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2)  # -> (B, 16, 4, 72)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2)  # -> (B, 32, 2, 36)

        # Fully connected
        self.fc = nn.Linear(32 * 2 * 36, self.num_outputs)

    def forward(self, x):
        # Input: (B, 6, 9, 144) → CNN requires channel first: (B, C, H, W)
        x = x.permute(0, 1, 2, 3)  # Already correct if input shape is (B, 6, 9, 144)
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return torch.sigmoid(x)  # Multi-label binary output

## **Validation**

In [16]:
from torch.utils.data import random_split

total_len = len(dataset)
val_len = int(0.2 * total_len)
train_len = total_len - val_len

train_dataset, val_dataset = random_split(dataset, [train_len, val_len])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score

def compute_metrics(y_true, y_pred, threshold=0.5):
    y_pred_bin = (y_pred >= threshold).astype(int)
    precision = precision_score(y_true, y_pred_bin, average='samples', zero_division=0)
    recall = recall_score(y_true, y_pred_bin, average='samples', zero_division=0)
    f1 = f1_score(y_true, y_pred_bin, average='samples', zero_division=0)
    return precision, recall, f1

In [19]:
def validate(model, val_loader, device):
    model.eval()
    all_preds = []
    all_targets = []
    val_loss = 0
    loss_fn = nn.BCELoss()

    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            outputs = model(xb)
            loss = loss_fn(outputs, yb)
            val_loss += loss.item() * xb.size(0)

            all_preds.append(outputs.cpu().numpy())
            all_targets.append(yb.cpu().numpy())

    val_loss /= len(val_loader.dataset)
    all_preds = np.vstack(all_preds)
    all_targets = np.vstack(all_targets)

    precision, recall, f1 = compute_metrics(all_targets, all_preds)

    return val_loss, precision, recall, f1


## **Train**

In [13]:
# 🔧 Modeli oluştur
model = TabCNN(num_strings=6, num_frets=19)

# 💻 Cihaz ayarı (Colab GPU varsa otomatik kullanılır)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 🎯 Loss ve optimizer
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
def train_one_epoch(model, loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0

    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        outputs = model(xb)
        loss = loss_fn(outputs, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * xb.size(0)

    return total_loss / len(loader.dataset)

In [20]:
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn, device)
    val_loss, precision, recall, f1 = validate(model, val_loader, device)
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1: {f1:.4f}")

Epoch 1/10 | Train Loss: 0.0074 | Val Loss: 0.0061 | Precision: 0.7552 | Recall: 0.7426 | F1: 0.7413
Epoch 2/10 | Train Loss: 0.0065 | Val Loss: 0.0066 | Precision: 0.7630 | Recall: 0.7799 | F1: 0.7633
Epoch 3/10 | Train Loss: 0.0058 | Val Loss: 0.0057 | Precision: 0.7708 | Recall: 0.7470 | F1: 0.7515
Epoch 4/10 | Train Loss: 0.0065 | Val Loss: 0.0072 | Precision: 0.6979 | Recall: 0.6671 | F1: 0.6753
Epoch 5/10 | Train Loss: 0.0050 | Val Loss: 0.0051 | Precision: 0.7552 | Recall: 0.7266 | F1: 0.7328
Epoch 6/10 | Train Loss: 0.0056 | Val Loss: 0.0051 | Precision: 0.7786 | Recall: 0.7582 | F1: 0.7623
Epoch 7/10 | Train Loss: 0.0052 | Val Loss: 0.0054 | Precision: 0.7839 | Recall: 0.7721 | F1: 0.7717
Epoch 8/10 | Train Loss: 0.0042 | Val Loss: 0.0058 | Precision: 0.7474 | Recall: 0.7227 | F1: 0.7286
Epoch 9/10 | Train Loss: 0.0038 | Val Loss: 0.0051 | Precision: 0.7652 | Recall: 0.7504 | F1: 0.7483
Epoch 10/10 | Train Loss: 0.0035 | Val Loss: 0.0053 | Precision: 0.7769 | Recall: 0.7852 | 